# Calibrate rigid wage

In [1]:
using NBInclude
@nbinclude("startup.ipynb")

Startup run in 7.133554935455322 secs.


In [2]:
function estimate(simpar::SimPar;
        x0 = [13.459713595432413, 0.0, 0.06874436711655102, 8.285702150626136, 2.44522060308835, 0.1755513264749144],
        lbd= [1, 0, 0, 0, 0, 0], 
        ubd = [200, 100, 1, 100, 10, 10],
        algo = :LN_NELDERMEAD, ## algo :LN_NELDERMEAD :LN_SBPLX :LN_BOBYQA :LN_COBYLA
        maxti=7*60*60, maxev=2000, xtol=1e-2,
        verbose=false) ## true shows the distance at each evaluation)

    Npar=length(x0)
    opt = NLopt.Opt(algo, Npar)      
    maxtime!(opt,maxti)
    maxeval!(opt,maxev)
    xtol_rel!(opt,xtol)
        
    ## bounds and free parameter    
    lower_bounds!(opt, lbd )
    upper_bounds!(opt, ubd)
    println("Optimisation with $Npar parameters,")
    println("   algo: $algo")
    println("   and maximum time: ", round(maxti/3600,digits=3))
    println("   and maximum eval: $maxev")        
    
    
    function obj(x,g) 
        a=16.
        out = try
                newp = initial_StrucPar_rigidwage(x[1], x[2], x[3], x[4], x[5], x[6])
                criterion(moments(simpar, newp), simpar.data)  
            catch
                Inf
            end
        return out
    end    
    
    nobs=0
    function obj_verbose(x,g)
        out = obj(x,g)       
        ## check every step
        step = 10
        if (nobs>1) & (div(nobs,step) * step == nobs)
            print("[$(round(x[1],sigdigits=2)),$(round(x[2],sigdigits=2)),$(round(x[3],sigdigits=2))]")
            print(":$(round(out,sigdigits=4))// ")
        end
        nobs += 1
        return(out)
    end
    
    min_objective!(opt,obj)
    verbose ? min_objective!(opt,obj_verbose) : ()

    println("Initial point: $(x0)")
    (minf,minvec,ret) = NLopt.optimize(opt,x0)
    println("The minimum is reached at ",minf, " with ", ret)
    println("  $minvec")
    return minvec
end

println("One evaluation takes:")
@time let
        newp = initial_StrucPar_rigidwage(13.459713595432413, 0.0, 0.06874436711655102, 8.285702150626136, 2.44522060308835, 0.1755513264749144)
        criterion(moments(simpar, newp), simpar.data) 
end

One evaluation takes:
 18.916099 seconds (11.12 M allocations: 12.251 GiB, 6.83% gc time, 39.21% compilation time)


0.2700233370192475

In [3]:
@time estimate(simpar)

Optimisation with 6 parameters,
   algo: LN_NELDERMEAD
   and maximum time: 7.0
   and maximum eval: 2000
Initial point: [13.459713595432413, 0.0, 0.06874436711655102, 8.285702150626136, 2.44522060308835, 0.1755513264749144]
/!\ Maximum number of iterations reached.
/!\ Maximum number of iterations reached.
/!\ Maximum number of iterations reached.
/!\ Maximum number of iterations reached.
/!\ Maximum number of iterations reached.
/!\ Maximum number of iterations reached.
/!\ Maximum number of iterations reached.
/!\ Maximum number of iterations reached.
The minimum is reached at 0.2700233370192475 with XTOL_REACHED
  [13.459713595432413, 0.0, 0.06874436711655102, 8.285702150626136, 2.44522060308835, 0.1755513264749144]
2064.889716 seconds (682.96 M allocations: 2.178 TiB, 8.68% gc time, 0.01% compilation time)


6-element Vector{Float64}:
 13.459713595432413
  0.0
  0.06874436711655102
  8.285702150626136
  2.44522060308835
  0.1755513264749144